<h1><center>A/B Testing - General flow and guidelines</center></h1>

<b>Typical flow for performing A/B testing should follow these steps:</b>
1. Think about the business question.  What are you testing? It might be difficult to tease out, but one should be clear about defining what's the minimum difference or margin between **A** and **B** that is meaningful for the product manager.   
3. Using the current CTR (or another metric in question) and the minimum margin chosen, perform sample size calculation to get an educated estimate of traffic required in order to conclude the test with confidence.
4. Given the sample size calculations, decide upon an appropriate sampling plan to recruit candidates in each arm of your study. Typically, the population of users is heteregenous in terms of characteristics that can influence the study, and care should be taken that both the arms of the study have _similar_ users (similar w.r.t characteristics that can influence the outcome of the study, e.g. engagement profile). **One** way to ensure a fair distribution of users in each arm is to first group all possible users with respect to certain quantifiable metric (preferably the outcome measure we seek to optimize) and then do cluster sampling to recruit users for each arm of the study.
5. Launch the study and collect data. The exeperimenter should strive to collect all the data as per the sampling plan. Since the difference in the groups **A** and **B** is not guranteed to evolve monotonically, stopping the test early can significantly increase the chances of drawing inaccurate conclusions.
5. After the data has been collected, perform the hypothesis testing using appropriate statistic.  Typically z-statistic is used by approximating the underlying Binomial distribution with a Gaussian via continuity correction.
6. Before drawing inference from the results (the statistic value and the corresponding p-value), review the assumptions made in the planning (design of experiments) and execution (data collection and monitoring) phases of the study. For example: 
    * _Is the CTR too low making the estimate of standard error unreliable which might inflate our confidence in our results_?
    * _Is the sample size large enough to justify approximating the Binomial with a Gaussian_?
    * _Did we do a good job of ensuring that **A** and **B** had similar distributions of users to justify choice of pooled standard deviation calculation_?
7. Interpret the results of your hypothesis testing and translate it back to the business question you started with.

In [36]:
# Manage Imports
import os
import numpy
from scipy.stats import norm
import numpy as np
from scipy.special import betaln
from __future__ import print_function

#### We have been asked to answer the business question if a variant/experiment **A** can help us increase our CTR by an amount X (two cases considered below: X=0.5% and X=1.0%) as compared to **B**, the control.  
#### Given this objective we will have to design our experiment such that it is sensitive enough to detect these changes (lift), i.e. choose appropriate sample size per arm of the study.
#### First we make some decisions about our experiment regarding how much of Type I (alpha) and Type II (beta) errors we are comfortable with, and then proceed with sample size calculations.


In [2]:
# Some assumptions for performing analysis on data
min_detectable_diff_1 = 0.005 #i.e. ctr lift of 0.5%
min_detectable_diff_2 = 0.01 #i.e. ctr lift of 1%
alpha_level = 0.05 #If I detect the minimum difference/gain, there should be at least 95% chance that it actually exists
beta_level = 0.2 #I should be able to detect the minimum difference/gain I am looking for at least 80% of the time, if it exists 

min_detectable_diff_1_text = str(min_detectable_diff_1 * 100) + '%'
min_detectable_diff_2_text = str(min_detectable_diff_2 * 100) + '%'
#print(min_acceptable_lift_1_text, min_acceptable_lift_2_text)

In [31]:
# Sample size calculation given our assumptions
base_ctr = 0.05 #5% CTR
z_score_half_alpha = norm.ppf(1-alpha_level/2) #sample size should be carried out with half the alpha, even when you intend to do one sided test
z_score_beta = norm.ppf(1-beta_level)
#print(z_score_half_alpha, z_score_beta)

# Sample size for case 1 (using min_detectable_diff_1: 0.005)
p1 = base_ctr
p2 = base_ctr + min_detectable_diff_1
A = (z_score_half_alpha + z_score_beta)**2
B = (p1*(1-p1)+p2*(1-p2))/(p1-p2)**2
sample_size_case_1 = int(np.ceil(A * B))
print('Sample size to detect a minimum lift of', min_detectable_diff_1_text,'in CTR =', sample_size_case_1,'in each group')

# Sample size for case 2 (using min_detectable_diff_2: 0.01)
p1 = base_ctr
p2 = base_ctr + min_detectable_diff_2
A = (z_score_half_alpha + z_score_beta)**2
B = (p1*(1-p1)+p2*(1-p2))/(p1-p2)**2
sample_size_case_2 = int(np.ceil(A * B))
print('Sample size to detect a minimum lift of', min_detectable_diff_2_text,'in CTR =', sample_size_case_2,'in each group')

Sample size to detect a minimum lift of 0.5% in CTR = 31231 in each group
Sample size to detect a minimum lift of 1.0% in CTR = 8155 in each group


#### As seen from results above, larger sample size is needed if we wish to detect even small changes in CTR.  Essentially, the larger the sample size per arm of the study, the more sensitivie the study is.
<br></br> 
#### Now assume the data was collected without any hickups, and we have the required information to do hypothesis testing.  The data looks like as follows:

In [5]:
# Set up data for analysis
num_ctrl = 1657
num_ctrl_views = 14272
num_ctrl_clicks = 1123
ctr_ctrl = num_ctrl_clicks / float(num_ctrl_views)

num_exp = 1610
num_exp_views = 12644
num_exp_clicks = 1085
ctr_exp = num_exp_clicks / float(num_exp_views)

print('ctr_ctrl =',ctr_ctrl,'  AND ctr_exp =',ctr_exp)

ctr_ctrl = 0.0786855381166   AND ctr_exp = 0.0858114520721


#### Reviewing some assumptions:
* We will assume that we did a fabulous job (using the module being built by Tenzin and Larry) for user selection for each arm of the study and the two arms have near-identical distribution of users.
* Clearly we have more than enough samples collected in each group to justify approximating Binomial with a Gaussian with continuity correction.  Hence we will be using z-statistic to do our statistical testing.
* Neither of the ratios _ctr__ctrl_ or _ctr__exp_ are too small to give unreliable estimate of the standard error.
* Since we did not collect enough number of samples as per our sampling plan, out test is not sensitive enough to detect lift of 0.5%. However, the test should be sensitive enough to detect lift of 1% reliably should that lift exist.

**Performing the hypothesis testing:**

In [32]:
# Analyze the data collected (using continuity correction: approximating binomial with normal distribution)
p1 = ctr_ctrl
p2 = ctr_exp
n1 = num_ctrl_views
n2 = num_exp_views
pooled_std_error = np.sqrt((p1*(1-p1)/n1) + (p2*(1-p2)/n2))
print('pooled std error = ', pooled_std_error)

min_acceptable_lift_1 = 0.005 #lift of 0.5%
min_acceptable_lift_2 = 0.01  #lift of 1.0%

min_acceptable_lift_1_text = str(min_acceptable_lift_1 * 100) + '%'
min_acceptable_lift_2_text = str(min_acceptable_lift_2 * 100) + '%'

pooled std error =  0.0033591395578084927


#### Question being asked: Can I expect to get ANY lift given my data (i.e. are the two proportions different)?

In [15]:
z_score = (p2-p1) / pooled_std_error
p_value = round(1-norm.cdf(z_score),4)
z_score = round(z_score, 4)
print('z_score =',z_score, ' AND, corresponding p-value =',p_value)
prob_of_success_as_percent = str((1-p_value)*100)+'%'
print('\nConclusion: There is',prob_of_success_as_percent,'chance that given the data '+
      'we will notice a lift in CTR if go with experiment instead of control')

z_score = 2.1214  AND, corresponding p-value = 0.0169

Conclusion: There is 98.31% chance that given the data we will notice a lift in CTR if go with experiment instead of control


#### Question being asked: Can I expect to get a lift of min_acceptable_lift_1 (=0.5%) given my data?

In [16]:
z_score_1 = ((p2-p1) - min_acceptable_lift_1) / pooled_std_error
p_value_z1 = round(1-norm.cdf(z_score_1),4)
z_score_1 = round(z_score_1, 4)
print('z_score =',z_score_1, ' AND, corresponding p-value =',p_value_z1)
prob_of_success_as_percent = str((1-p_value_z1)*100)+'%'
print('\nConclusion: There is',prob_of_success_as_percent,'chance that given the data '+
      'we will notice at least a lift of',min_acceptable_lift_1_text,'in CTR if go with experiment instead of control')

z_score = 0.6329  AND, corresponding p-value = 0.2634

Conclusion: There is 73.66% chance that given the data we will notice at least a lift of 0.5% in CTR if go with experiment instead of control


In [23]:
### Question being asked: Can I expect to get a lift of min_acceptable_lift_2 (=0.1%) given my data?
z_score_2 = ((p2-p1) - min_acceptable_lift_2) / pooled_std_error
p_value_z2 = round(1-norm.cdf(z_score_2),4)
z_score_2 = round(z_score_2, 4)
print('z_score =',z_score_2, ' AND, corresponding p-value =',p_value_z2)
prob_of_success_as_percent = str((1-p_value_z2)*100)+'%'
print('\nConclusion: There is',prob_of_success_as_percent,'chance that given the data '+
      'we will notice at least a lift of',min_acceptable_lift_2_text,'in CTR if go with experiment instead of control')

z_score = -0.8556  AND, corresponding p-value = 0.8039

Conclusion: There is 19.61% chance that given the data we will notice at least a lift of 1.0% in CTR if go with experiment instead of control


In [21]:
# Confidence Interval of experiment
std_error_exp = np.sqrt(ctr_exp*(1 - ctr_exp)/num_exp_views)
print(std_error_exp)
z = norm.ppf(0.95)
conf_int_exp_lower, conf_int_exp_upper = ctr_exp-z*std_error_exp, ctr_exp+z*std_error_exp
print('Confidence Interval for Experiment: ',(round(conf_int_exp_lower,4), round(conf_int_exp_upper, 4)))

0.002490853987671606
Confidence Interval for Experiment:  (0.0817, 0.0899)


In [20]:
# Confidence Interval of control
std_error_ctrl = np.sqrt(ctr_ctrl*(1 - ctr_ctrl)/num_ctrl_views)
print(std_error_ctrl)
z = norm.ppf(0.95)
conf_int_ctrl_lower, conf_int_ctrl_upper = ctr_ctrl-z*std_error_ctrl, ctr_ctrl+z*std_error_ctrl
print('Confidence Interval for Control: ',(round(conf_int_ctrl_lower,4), round(conf_int_ctrl_upper,4)))

0.00225376684262909
Confidence Interval for Control:  (0.075, 0.0824)


#### Interpretting results of the experiment.


Given the results, we can be confident that there will be a +ve lift if we go with **A** instead of **B**. 

**Conclusion about 1% absolute lift**:  Our experiment was setup/executed to be sensitive enough to confidently detect a lift of 1.0% (absolute) if the lift existed. Given that we observed only about 19% chance that the lift would be at least 1%, we can conclude that 1% absolute lift is out of reach with option **A**.  

**Conclusion about 0.5% absolute lift**:  The experiment was not executed to be sensitive enough to detect absolute lift of 0.5%.  The results indicate that, we can state with approximately 73% confidence that the lift will be at least 0.5% (absolute). This confidence is not high enough to offset other sources of uncertainty not explicitly accounted for in the experiment.  However, given that we did not drive our experiment to completion as per plan (stopped short), there is a chance that we might have seen statistically significant results if we had completed the experiment. 

#### Now, let's say the PM asks: how much of a lift can I expect with 95% confidence given the data I have collected?

**Solution**:  The critical z-value for 95% confidence interval = 1.96.
Hence maximum lift we can detect with 95% confidence given the data = (p_exp - p_ctrl) - (1.96 * pooled_std_error)
 Which ends up being:

In [35]:
max_lift_to_expect_95percent = (p2-p1) - (1.96 * pooled_std_error)
print('Maximum absolute lift we can expect (with 95% confidence) = ', str(round(max_lift_to_expect_95percent * 100, 2))+'%')
print('Maximum relative lift we can expect (with 95% confidence) = ', str(round(max_lift_to_expect_95percent/base_ctr * 100, 2))+'%')

Maximum absolute lift we can expect (with 95% confidence) =  0.05%
Maximum relative lift we can expect (with 95% confidence) =  1.08%


### What if there was no sample size calculations done?  I have been collecting data and am just curious if I should continue the test.  This situation can arise for multiple reasons, one of which can be that there is reasonable way of getting "base_ctr" i.e. no historic data on the baseline.  Basically I just want to compare two types of experiments. 

### One approach could be that I can assume a base_ctr and proceed for one of them, but which one??  Typically in these scenarios a more robust way of proceeding is:
###    1) Assume that the probabilities of clicks / no clicks come from a known parametric distribution (often Beta family), and get a prior, 
###    2) Calculate the posterior probability of a particular variant (say B) winning as compared to another variant (say A).


** VERY GOOD REFERENCE: https://www.evanmiller.org/bayesian-ab-testing.html **

In [37]:
def calc_prob_clickB_greaterthan_clickA(clicks_A, total_A, clicks_B, total_B):
    '''
    :param clicks_A: pos integer: number of clicks A got
    :param total_A: pos integer: number of times A was presented, i.e. total number of times A had a chance of
                    getting clicked.
    :param clicks_B: pos integer: number of clicks B got
    :param total_B: pos integer: number of times B was presented, i.e. total number of times B had a chance of
                    getting clicked.
    :return: probability that B would be clicked more number of times than A.
    '''
    assert(clicks_A * clicks_B * total_A * total_B > 0) # All must be positive.  Not checking for integer, being lazy.

    posA, negA = clicks_A + 1 , total_A - clicks_A + 1
    posB, negB = clicks_B + 1, total_B - clicks_B + 1

    prob_B_wins = 0
    for i in range(posB-1):
        log_numerator = betaln(posA+i, negB + negA)
        log_denominator = np.log(negB+i) + betaln(1+i, negB) + betaln(posA, negA)
        prob_B_wins += np.exp(log_numerator - log_denominator)
    return prob_B_wins

#### Using data from our previous eample:

In [38]:
num_A_views = 14272   
num_A_clicks = 1123

num_B_views = 12644  
num_B_clicks = 1085

In [40]:
prob_of_B_winning_over_A = calc_prob_clickB_greaterthan_clickA(num_A_clicks, num_A_views, num_B_clicks, num_B_views)
print('Prob that experiment B will win over experiment A =', round(prob_of_B_winning_over_A,4))

Prob that experiment B will win over experiment A = 0.9823
